# OpenAI Simple Chat Model

In [1]:
# ! pip install openai

Load a generic webshop goal

In [4]:
from agentquest.benchmarks.webshop import WebshopDriver, WebshopUtils
from dotenv import load_dotenv
import os

load_dotenv()

# proxy_settings(set=False)
WebshopUtils.test_interface_api()

http://localhost:5555
2025-01-07 15:27 - INFO - Connection test to interface successful.
{
  "test": "okay"
}



Initialize the prompt and the LLM

In [5]:
from openai import OpenAI

client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
)

# Initalize the system prompt
messages = [
    {
        "role": "developer",
        "content": "You are a web shopping agent. Follow the instructions provided.",
    },
]

Initialize the Webshop driver and get the first observation. Reset method calls automatically loads a task.

In [6]:
driver = WebshopDriver(base_url="http://localhost:5555")
obs = driver.reset()
print(obs.output)

Welcome to WebShop, a simulated e-commerce website environment. Your goal is to find and buy the best product matching the instruction provided.
There are two possible actions `search[<query>]` and `click[<clickable>]`. These actions will help you navigate through the WebShop environment. The list of possible actions and clickables are provided in the observation output after every action.
Your output should in the following format:
Action: <action>
WebShop [SEP] Instruction: [SEP] Find me slim fit, machine wash women's jumpsuits, rompers & overalls with short sleeve, high waist, polyester spandex for daily wear with color: green stripe, and size: large, and price lower than 50.00 dollars [SEP] Search
Available actions in json:
{"clickables": ["search"], "has_search_bar": true}


Start the LLM loop solving the task

In [7]:
step_cnt = 0
max_steps = 10
while not obs.success and obs.can_proceed and step_cnt < max_steps:
    # Provide the observation to the LLM
    messages.append({"role": "user", "content": obs.output})

    # Get the LLM output
    agent_choice = (
        client.chat.completions.create(model="gpt-4o-mini", messages=messages)
        .choices[0]
        .message.content
    )

    # Provide the choice to the LLM as history
    messages.append({"role": "assistant", "content": agent_choice})
    print(agent_choice)

    # Build the action
    obs = driver.step_raw(agent_choice)
    print(obs.output)

    step_cnt += 1

Action: search[slim fit, machine wash women's jumpsuits, rompers & overalls with short sleeve, high waist, polyester spandex for daily wear green stripe large under 50.00 dollars]
WebShop [SEP] Instruction: [SEP] Find me slim fit, machine wash women's jumpsuits, rompers & overalls with short sleeve, high waist, polyester spandex for daily wear with color: green stripe, and size: large, and price lower than 50.00 dollars [SEP] Search
Available actions in json:
{"clickables": ["search"], "has_search_bar": true}
Instruction: [SEP] Find me slim fit, machine wash women's jumpsuits, rompers & overalls with short sleeve, high waist, polyester spandex for daily wear with color: green stripe, and size: large, and price lower than 50.00 dollars [SEP] Back to Search [SEP] Page 1 (Total results: 50) [SEP] Next > [SEP] B099WX3CV5 [SEP] Women Aesthetic Short Sleeve Jumpsuit Bodycon Sexy V Neck Button Shorts Rompers Knitted One Piece Bodysuit Overall [SEP] $13.99 to $24.89 [SEP] B09QCVCYVY [SEP] Burb

In [8]:
driver.metrics.export()

2025-01-07 15:28 - WARNING - Exception encountered: Metrics.get_repetition_rate() missing 1 required positional argument: 'num_execution_steps'. Unable to calculate repetition rate.


{'goal': "WebShop [SEP] Instruction: [SEP] Find me slim fit, machine wash women's jumpsuits, rompers & overalls with short sleeve, high waist, polyester spandex for daily wear with color: green stripe, and size: large, and price lower than 50.00 dollars [SEP] Search",
 'success': False,
 'actions': [{'value': "search[slim fit, machine wash women's jumpsuits, rompers & overalls with short sleeve, high waist, polyester spandex for daily wear green stripe large under 50.00 dollars]"},
  {'value': 'click[b099wx3cv5]'},
  {'value': 'click[buy now]'}],
 'states': [{'value': {'available_actions': {'clickables': ['back to search',
      'next >',
      'b099wx3cv5',
      'b09qcvcyvy',
      'b099231v35',
      'b09nds8f4v',
      'b09pbpz24z',
      'b09qqp3356',
      'b09m63b87v',
      'b09q37jqz6',
      'b09q5zhrvm',
      'b09r9ycm6r'],
     'has_search_bar': False},
    'done': False,
    'info': None,
    'obs': "Instruction: [SEP] Find me slim fit, machine wash women's jumpsuits, rom